In [ ]:
%%shell

# Install pycocotools
git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi/PythonAPI
python setup.py build_ext install

fatal: destination path 'cocoapi' already exists and is not an empty directory.
running build_ext
skipping 'pycocotools/_mask.c' Cython extension (up-to-date)
running install
running bdist_egg
running egg_info
writing pycocotools.egg-info/PKG-INFO
writing dependency_links to pycocotools.egg-info/dependency_links.txt
writing requirements to pycocotools.egg-info/requires.txt
writing top-level names to pycocotools.egg-info/top_level.txt
writing manifest file 'pycocotools.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.6/pycocotools/_mask.cpython-36m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.6/pycocotools/__init__.py -> build/bdist.linux-x86_64/egg/pycocotools
copying build/lib.linux-x86_64-3.6/pycocotools/coco.py -> build/bdist.linux-x86_64/egg/pycoc

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import xml.etree.ElementTree as ET

class WT(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "annotation_color"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "annotation_color", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        masks = mask == obj_ids[:, None, None]
        num_objs = len(obj_ids)
        boxes = []

        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from zipfile import ZipFile 
  
file = '/content/drive/My Drive/piscinacaixadaguacolors.zip'

with ZipFile(file, 'r') as zip:
  print('Extracting all the files now...') 
  zip.extractall() 
  print('Done!') 


Extracting all the files now...
Done!


In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator


backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280
anchor_generator = AnchorGenerator(sizes=((20,30,35),),
                                   aspect_ratios=((1.0),))
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names='0',
                                                output_size=7,
                                                sampling_ratio=2)
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler,
                   min_size=1280)

In [ ]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

fatal: destination path 'vision' already exists and is not an empty directory.
HEAD is now at be37608 version check against PyTorch's CUDA version


In [ ]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T


def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
dataset = WT('piscinacaixadaguacolors/wtanks/train', get_transform(train=True))
dataset_test = WT('piscinacaixadaguacolors/wtanks/test', get_transform(train=False))

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=4, shuffle=True, num_workers=4,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


num_classes = 2

model.to(device)

params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr=0.0001)

# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [ ]:
import time

num_epochs = 50

start1 = time.time()

for epoch in range(num_epochs):

    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=20)

    #lr_scheduler.step()

    #evaluate(model, data_loader_test, device=device)

    #torch.save(model.state_dict(), '/content/drive/My Drive/wt.pt')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Epoch: [0]  [  0/152]  eta: 0:21:55  lr: 0.000001  loss: 1.6007 (1.6007)  loss_classifier: 0.7590 (0.7590)  loss_box_reg: 0.0102 (0.0102)  loss_objectness: 0.7361 (0.7361)  loss_rpn_box_reg: 0.0955 (0.0955)  time: 8.6543  data: 7.2845  max mem: 8164
Epoch: [0]  [ 20/152]  eta: 0:05:19  lr: 0.000014  loss: 0.9372 (1.2788)  loss_classifier: 0.2782 (0.3635)  loss_box_reg: 0.0053 (0.0062)  loss_objectness: 0.5299 (0.5422)  loss_rpn_box_reg: 0.0657 (0.3670)  time: 2.1054  data: 1.3852  max mem: 9145
Epoch: [0]  [ 40/152]  eta: 0:04:05  lr: 0.000027  loss: 0.5883 (1.3555)  loss_classifier: 0.2234 (0.3044)  loss_box_reg: 0.0132 (0.0097)  loss_objectness: 0.2741 (0.4399)  loss_rpn_box_reg: 0.0872 (0.6015)  time: 1.9599  data: 1.2124  max mem: 9197
Epoch: [0]  [ 60/152]  eta: 0:03:34  lr: 0.000040  loss: 0.4956 (1.2745)  loss_classifier: 0.1752 (0.2646)  loss_box_reg: 0.0210 (0.0129)  loss_objectness: 0.2382 (0.3851)  loss_rpn_box_reg: 0.0733 (0.6118)  time: 2.6183  data: 1.8752  max mem: 9197


In [ ]:
end1 = time.time()
print("training time =", end1-start1)	

start2 = time.time()
evaluate(model, data_loader_test, device=device)
end2 = time.time()
print("evaluate time =", end2-start2)

torch.save(model.state_dict(), '/content/drive/My Drive/wtfinal2.pt')


training time = 16844.667036771774
creating index...
index created!


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


Test:  [  0/148]  eta: 0:02:56  model_time: 0.0842 (0.0842)  evaluator_time: 0.1333 (0.1333)  time: 1.1907  data: 0.9471  max mem: 9242
Test:  [100/148]  eta: 0:00:10  model_time: 0.0581 (0.0576)  evaluator_time: 0.0567 (0.0827)  time: 0.1876  data: 0.0302  max mem: 9242
Test:  [147/148]  eta: 0:00:00  model_time: 0.0573 (0.0595)  evaluator_time: 0.0650 (0.0812)  time: 0.1999  data: 0.0231  max mem: 9242
Test: Total time: 0:00:31 (0.2124 s / it)
Averaged stats: model_time: 0.0573 (0.0595)  evaluator_time: 0.0650 (0.0812)
Accumulating evaluation results...
DONE (t=0.05s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.341
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.758
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.226
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.344
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.204
 Aver